In [ ]:
import os
from dotenv import load_dotenv

load_dotenv("../../config/.env")
os.environ["LANGSMITH_TRACING"] = "true"

In [17]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("deepseek-chat", model_provider="deepseek")

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

class Classification(BaseModel):
    sentiment: str = Field(description="所提供文本的情绪")
    aggressiveness: int = Field(description="文本的攻击性在 1 到 10 的量表上是多少")
    language: str = Field(description="所提供文本的语言")

tagging_prompt = ChatPromptTemplate.from_template(
    """
    从以下段落中提取信息。
    只提取Classification中提到的属性。
    段落：
    {input}
    """
)

inp_1 = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
inp_2 = "Estoy muy enojado con vos! Te voy a dar tu merecido!"

prompt_1 = tagging_prompt.invoke({"input": inp_1})
prompt_2 = tagging_prompt.invoke({"input": inp_2})
structured_llm = llm.with_structured_output(Classification)
response_1 = structured_llm.invoke(prompt_1)
response_2 = structured_llm.invoke(prompt_2)

print(response_1.model_dump())
print(response_2.model_dump())

{'sentiment': 'positive', 'aggressiveness': 1, 'language': 'Spanish'}
{'sentiment': 'enojado', 'aggressiveness': 8, 'language': 'es'}


In [19]:
class ClassificationFinerControl(BaseModel):
    sentiment: str = Field(..., enum=["happy", "sad", "neutral"])
    aggressiveness: int = Field(
        ...,
        description="描述该陈述的攻击性程度，数字越高，攻击性越强",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(..., enum=["中文", "英文", "西班牙语"])

structured_finer_control_llm = llm.with_structured_output(ClassificationFinerControl)
response_finer_control_1 = structured_finer_control_llm.invoke(prompt_1)
response_finer_control_2 = structured_finer_control_llm.invoke(prompt_2)

print(response_finer_control_1)
print(response_finer_control_2)

sentiment='happy' aggressiveness=1 language='西班牙语'
sentiment='sad' aggressiveness=5 language='西班牙语'
